In [37]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
nltk.download('stopwords')
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from sklearn.manifold import TSNE

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rutuj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20001 entries, 0 to 20000
Data columns (total 3 columns):
Unnamed: 0    20001 non-null int64
content       20001 non-null object
label         20001 non-null int64
dtypes: int64(2), object(1)
memory usage: 468.9+ KB


In [58]:
df[df.label==0].head()

,Unnamed: 0,content,label
7822,7822,yeah I got 2 backups for all that. I just hate...,0
7823,7823,I hate using my BB but love my iPhone. Haven'...,0
7824,7824,wow lol sounds like a lot of piss then hehehe,0
7825,7825,not a damn thang..the typical rap beef. one pe...,0
7826,7826,well damn!! where have you been when i have ne...,0


In [60]:
df[df.label==1].head()

,Unnamed: 0,content,label
0,0,Get fucking real dude.,1
1,1,She is as dirty as they come and that crook ...,1
2,2,why did you fuck it up. I could do it all day...,1
3,3,Dude they dont finish enclosing the fucking s...,1
4,4,WTF are you talking about Men? No men thats n...,1


In [61]:
import re
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

df['content'] = df['content'].map(lambda x: clean_text(x))

In [62]:
### Create sequence
vocabulary_size = 1000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])
data = pad_sequences(sequences, maxlen=10)

In [63]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=10))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
    return model_conv
model_conv = create_conv_model()
model_conv.fit(data, df['label'], validation_split=0.2, epochs = 10)

Train on 16000 samples, validate on 4001 samples
Epoch 1/10
16000/16000 [==============================] - 8s 495us/step - loss: 0.6670 - acc: 0.5770 - val_loss: 0.6504 - val_acc: 0.8113
Epoch 2/10
16000/16000 [==============================] - 6s 391us/step - loss: 0.5311 - acc: 0.7066 - val_loss: 0.5421 - val_acc: 0.4164
Epoch 3/10
16000/16000 [==============================] - 7s 418us/step - loss: 0.3825 - acc: 0.7862 - val_loss: 0.5477 - val_acc: 0.8630
Epoch 4/10
16000/16000 [==============================] - 6s 404us/step - loss: 0.3085 - acc: 0.8245 - val_loss: 0.5744 - val_acc: 0.8770
Epoch 5/10
16000/16000 [==============================] - 6s 405us/step - loss: 0.2787 - acc: 0.8343 - val_loss: 0.6026 - val_acc: 0.9008
Epoch 6/10
16000/16000 [==============================] - 7s 410us/step - loss: 0.2661 - acc: 0.8427 - val_loss: 0.6239 - val_acc: 0.8948
Epoch 7/10
16000/16000 [==============================] - 7s 408us/step - loss: 0.2497 - acc: 0.8495 - val_loss: 0.7518 - v